In [8]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
sys.path.append("/n/home12/binxuwang/Github/DiffusionReasoning/RPM_data_generation")
sys.path.append("/n/home12/binxuwang/Github/DiffusionReasoning")

In [21]:
import numpy as np 
import torch 
import os 
import pickle
import random 
import time 
from os.path import join
from tqdm import tqdm, trange
os.environ['CUDA_VISIBLE_DEVICES']='0'
device = torch.device('cpu')

In [10]:
from utils_dataset import sample_nan, fill_panel, sample_nan_pos, sample_x_with_num

In [ ]:
data_dir = '/n/home12/binxuwang/Github/DiffusionReasoning/rule_data/20240308_data/'


In [17]:

# Load numpy arrays
# Load pickled dictionaries
r_all_M7 = np.load(join(data_dir, 'r_list_M7.npy'), allow_pickle=True)
r_dict_M7 = pickle.load(open(join(data_dir, 'r_dict_M7.pkl'), 'rb'))
r_last_M7 = pickle.load(open(join(data_dir, 'r_last_M7.pkl'), 'rb'))
r_all_M10 = np.load(join(data_dir, 'r_list_M10.npy'), allow_pickle=True)
r_dict_M10 = pickle.load(open(join(data_dir, 'r_dict_M10.pkl'), 'rb'))
r_last_M10 = pickle.load(open(join(data_dir, 'r_last_M10.pkl'), 'rb'))
r_list_pos = np.load(join(data_dir, 'r_list_pos.npy'), allow_pickle=True)
r_dict_pos = pickle.load(open(join(data_dir, 'r_dict_pos.pkl'), 'rb'))
r_list_num = np.load(join(data_dir, 'r_list_num.npy'), allow_pickle=True)
r_dict_num = pickle.load(open(join(data_dir, 'r_dict_num.pkl'), 'rb'))
r_dict_pos_withR = pickle.load(open(join(data_dir, 'r_dict_pos_withR.pkl'), 'rb'))

# Convert keys of r_dict_num to a list
r_dict_num_keylist = list(r_dict_num.keys())

In [18]:
num_samples_total = 1200500
num_samples_keep = 1200000
output_dir = '/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Datasets/RPM_dataset/RPM1000k'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(join(output_dir, 'aR'), exist_ok=True)

### Attribute 0 Shape

In [22]:
i_a = 0

for i_R in range(7): 
    
    start_time = time.time()
    
    r_list = r_all_M7[i_R]
    num_samples_per_r = int(np.ceil(num_samples_total/len(r_list)))

    attr_all = []

    for i_r in range(len(r_list)):

        for i_sample in trange(num_samples_per_r):

            # 1) x_shape is determined by the rule 
            x_shape = r_all_M7[i_R][i_r] # e.g. [[1], [1], [1]]

            potential_list = []
            while len(potential_list) == 0: 
                # 2) sample x_size and x_color attribute values to be not any rule ~ 
                # to avoid shortcut by just looking at the first two panels, make first2 same as some rule 
                x_size = sample_nan(r_all_M10, r_dict_M10, r_last_M10)
                x_color = sample_nan(r_all_M10, r_dict_M10, r_last_M10)

                # get the number of unique attribues per panel. This is the min for x_num
                num_attr_unique = [max([len(x_shape[i]), len(x_size[i]), len(x_color[i])]) for i in range(3)]

                # 3) sample number 
                # the way we sample it is to choose one 1)>x_min, 2) first 2 panel belong to some row 3) last panel not 
                potential_first2 = [k for k in r_dict_num_keylist if int(k[0])>=num_attr_unique[0] and int(k[1])>=num_attr_unique[1]]
                if len(potential_first2) > 0: 
                    key_first2 = random.choice(potential_first2) # e.g., '77'
                    d = r_dict_num[key_first2]
                    potential_list = [i for i in range(num_attr_unique[2], 9) if i not in d]

            last = random.choice(potential_list) # could this give us no choice? 
            x_num = [[int(key_first2[0])], [int(key_first2[1])], [last]] # e.g., [7, 7, 4]
            x_pos = sample_nan_pos(x_num, r_dict_pos, r_dict_pos_withR) # 4) sample position 

            attr = np.asarray([fill_panel(x_pos[i], x_shape[i], x_size[i], x_color[i]) for i in range(3)])
            attr_all.append(attr)

            del x_shape, x_size, x_color, x_pos, x_num 

    dur = time.time() - start_time
    
    attr_all = np.asarray(attr_all)
    a = np.copy(attr_all).reshape((-1, 3*9*3))
    b = np.unique(a, axis=0)
    ind = np.random.choice(np.arange(len(attr_all)), num_samples_keep,replace=False)
    # np.save(output_dir+'aR/'+str(i_a)+'_'+str(i_R), attr_all[ind])
    np.save(join(output_dir, 'aR', f"{i_a}_{i_R}"), attr_all[ind])
    
    print(str(i_R)+' -- '+str(dur)+' -- '+str(len(b) == len(a)))

  0%|          | 0/200084 [00:00<?, ?it/s]

 21%|██        | 42246/200084 [00:10<00:38, 4048.95it/s]


KeyboardInterrupt: 

In [ ]:
i_a = 0 
for i_R in range(7, 10): 
    
    r_list = r_all_M7[i_R]
    ind_select = np.random.choice(len(r_list), num_samples_total, replace=False)
    
    attr_all = []
    start_time = time.time()
    
    for i_sample, i_r in tqdm(enumerate(ind_select)):

        # 1) x_shape is determined by the rule 
        x_shape = r_all_M7[i_R][i_r] # e.g. [[1], [1], [1]]

        potential_list = []
        while len(potential_list) == 0: 
            # 2) sample x_size and x_color attribute values to be not any rule ~ 
            # to avoid shortcut by just looking at the first two panels, make first2 same as some rule 
            x_size = sample_nan(r_all_M10, r_dict_M10, r_last_M10)
            x_color = sample_nan(r_all_M10, r_dict_M10, r_last_M10)

            # get the number of unique attribues per panel. This is the min for x_num
            num_attr_unique = [max([len(x_shape[i]), len(x_size[i]), len(x_color[i])]) for i in range(3)]

            # 3) sample number 
            # the way we sample it is to choose one 1)>x_min, 2) first 2 panel belong to some row 3) last panel not 
            potential_first2 = [k for k in r_dict_num_keylist if int(k[0])>=num_attr_unique[0] and int(k[1])>=num_attr_unique[1]]
            if len(potential_first2) > 0: 
                key_first2 = random.choice(potential_first2) # e.g., '77'
                d = r_dict_num[key_first2]
                potential_list = [i for i in range(num_attr_unique[2], 9) if i not in d]

        last = random.choice(potential_list) # could this give us no choice? 
        x_num = [[int(key_first2[0])], [int(key_first2[1])], [last]] # e.g., [7, 7, 4]
        x_pos = sample_nan_pos(x_num, r_dict_pos, r_dict_pos_withR) # 4) sample position 

        attr = np.asarray([fill_panel(x_pos[i], x_shape[i], x_size[i], x_color[i]) for i in range(3)])
        attr_all.append(attr)

        del x_shape, x_size, x_color, x_pos, x_num 
    
    dur = time.time() - start_time
    
    attr_all = np.asarray(attr_all)
    a = np.copy(attr_all).reshape((-1, 3*9*3))
    b = np.unique(a, axis=0)
    ind = np.random.choice(np.arange(len(attr_all)), num_samples_keep,replace=False)
    # np.save(output_dir+'aR/'+str(i_a)+'_'+str(i_R), attr_all[ind])
    np.save(join(output_dir, 'aR', f"{i_a}_{i_R}"), attr_all[ind])
    
    print(str(i_R)+' -- '+str(dur)+' -- '+str(len(b) == len(a)))

7 -- 3.9489974975585938 -- True
8 -- 3.9839329719543457 -- True
9 -- 3.8846371173858643 -- True


### Attribute 1 Size

In [ ]:
i_a = 1

for i_R in range(7): 
    
    start_time = time.time()
    
    r_list = r_all_M10[i_R]
    num_samples_per_r = int(np.ceil(num_samples_total/len(r_list)))

    attr_all = []

    for i_r in range(len(r_list)):

        for i_sample in trange(num_samples_per_r):

            # 1) x_shape is determined by the rule 
            x_size = r_list[i_r] # e.g. [[1], [1], [1]]

            potential_list = []
            while len(potential_list) == 0: 
                # 2) sample x_size and x_color attribute values to be not any rule ~ 
                # to avoid shortcut by just looking at the first two panels, make first2 same as some rule 
                x_shape = sample_nan(r_all_M7, r_dict_M7, r_last_M7)
                x_color = sample_nan(r_all_M10, r_dict_M10, r_last_M10)

                # get the number of unique attribues per panel. This is the min for x_num
                num_attr_unique = [max([len(x_shape[i]), len(x_size[i]), len(x_color[i])]) for i in range(3)]

                # 3) sample number 
                # the way we sample it is to choose one 1)>x_min, 2) first 2 panel belong to some row 3) last panel not 
                potential_first2 = [k for k in r_dict_num_keylist if int(k[0])>=num_attr_unique[0] and int(k[1])>=num_attr_unique[1]]
                if len(potential_first2) > 0: 
                    key_first2 = random.choice(potential_first2) # e.g., '77'
                    d = r_dict_num[key_first2]
                    potential_list = [i for i in range(num_attr_unique[2], 9) if i not in d]

            last = random.choice(potential_list) # could this give us no choice? 
            x_num = [[int(key_first2[0])], [int(key_first2[1])], [last]] # e.g., [7, 7, 4]
            x_pos = sample_nan_pos(x_num, r_dict_pos, r_dict_pos_withR) # 4) sample position 

            attr = np.asarray([fill_panel(x_pos[i], x_shape[i], x_size[i], x_color[i]) for i in range(3)])
            attr_all.append(attr)

            del x_shape, x_size, x_color, x_pos, x_num 

    dur = time.time() - start_time
    
    attr_all = np.asarray(attr_all)
    ind = np.random.choice(np.arange(len(attr_all)), num_samples_keep,replace=False)
    # np.save(output_dir+'aR/'+str(i_a)+'_'+str(i_R), attr_all[ind])
    np.save(join(output_dir, 'aR', f"{i_a}_{i_R}"), attr_all[ind])
    
    print(str(i_R)+' -- '+str(dur))

0 -- 3.9146580696105957
1 -- 3.8860762119293213
2 -- 3.9326653480529785
3 -- 3.932223320007324
4 -- 3.8784470558166504
5 -- 3.9110312461853027
6 -- 3.868302345275879


In [ ]:
i_a = 1 
for i_R in range(7, 10):
    
    r_list = r_all_M10[i_R]
    ind_select = np.random.choice(len(r_list), num_samples_total, replace=False)

    attr_all = []

    for i_sample, i_r in tqdm(enumerate(ind_select)):

        start_time = time.time()

        # 1) x_size is determined by the rule 
        x_size = r_list[i_r] # e.g. [[1], [1], [1]]

        potential_list = []
        while len(potential_list) == 0: 
            # 2) sample x_size and x_color attribute values to be not any rule ~ 
            # to avoid shortcut by just looking at the first two panels, make first2 same as some rule 
            x_shape = sample_nan(r_all_M7, r_dict_M7, r_last_M7)
            x_color = sample_nan(r_all_M10, r_dict_M10, r_last_M10)

            # get the number of unique attribues per panel. This is the min for x_num
            num_attr_unique = [max([len(x_shape[i]), len(x_size[i]), len(x_color[i])]) for i in range(3)]

            # 3) sample number 
            # the way we sample it is to choose one 1)>x_min, 2) first 2 panel belong to some row 3) last panel not 
            potential_first2 = [k for k in r_dict_num_keylist if int(k[0])>=num_attr_unique[0] and int(k[1])>=num_attr_unique[1]]
            if len(potential_first2) > 0: 
                key_first2 = random.choice(potential_first2) # e.g., '77'
                d = r_dict_num[key_first2]
                potential_list = [i for i in range(num_attr_unique[2], 9) if i not in d]

        last = random.choice(potential_list) # could this give us no choice? 
        x_num = [[int(key_first2[0])], [int(key_first2[1])], [last]] # e.g., [7, 7, 4]
        x_pos = sample_nan_pos(x_num, r_dict_pos, r_dict_pos_withR) # 4) sample position 

        attr = np.asarray([fill_panel(x_pos[i], x_shape[i], x_size[i], x_color[i]) for i in range(3)])
        attr_all.append(attr)

        del x_shape, x_size, x_color, x_pos, x_num 

    dur = time.time() - start_time
    
    attr_all = np.asarray(attr_all)
    a = np.copy(attr_all).reshape((-1, 3*9*3))
    b = np.unique(a, axis=0)
    ind = np.random.choice(np.arange(len(attr_all)), num_samples_keep,replace=False)
    # np.save(output_dir+'aR/'+str(i_a)+'_'+str(i_R), attr_all[ind])
    np.save(join(output_dir, 'aR', f"{i_a}_{i_R}"), attr_all[ind])
    
    print(str(i_R)+' -- '+str(dur)+' -- '+str(len(b) == len(a)))

7 -- 0.0003612041473388672 -- True
8 -- 0.0003108978271484375 -- True
9 -- 0.0003402233123779297 -- True


### Attribute 2 Color

In [ ]:
i_a = 2
for i_R in range(7): 
    
    start_time = time.time()
    
    r_list = r_all_M10[i_R]
    num_samples_per_r = int(np.ceil(num_samples_total/len(r_list)))

    attr_all = []

    for i_r in range(len(r_list)):

        for i_sample in trange(num_samples_per_r):

            # 1) x_color is determined by the rule 
            x_color = r_list[i_r] # e.g. [[1], [1], [1]]

            potential_list = []
            while len(potential_list) == 0: 
                # 2) sample x_size and x_color attribute values to be not any rule ~ 
                # to avoid shortcut by just looking at the first two panels, make first2 same as some rule 
                x_shape = sample_nan(r_all_M7, r_dict_M7, r_last_M7)
                x_size = sample_nan(r_all_M10, r_dict_M10, r_last_M10)

                # get the number of unique attribues per panel. This is the min for x_num
                num_attr_unique = [max([len(x_shape[i]), len(x_size[i]), len(x_color[i])]) for i in range(3)]

                # 3) sample number 
                # the way we sample it is to choose one 1)>x_min, 2) first 2 panel belong to some row 3) last panel not 
                potential_first2 = [k for k in r_dict_num_keylist if int(k[0])>=num_attr_unique[0] and int(k[1])>=num_attr_unique[1]]
                if len(potential_first2) > 0: 
                    key_first2 = random.choice(potential_first2) # e.g., '77'
                    d = r_dict_num[key_first2]
                    potential_list = [i for i in range(num_attr_unique[2], 9) if i not in d]

            last = random.choice(potential_list) # could this give us no choice? 
            x_num = [[int(key_first2[0])], [int(key_first2[1])], [last]] # e.g., [7, 7, 4]
            x_pos = sample_nan_pos(x_num, r_dict_pos, r_dict_pos_withR) # 4) sample position 

            attr = np.asarray([fill_panel(x_pos[i], x_shape[i], x_size[i], x_color[i]) for i in range(3)])
            attr_all.append(attr)

            del x_shape, x_size, x_color, x_pos, x_num 

    dur = time.time() - start_time
    
    attr_all = np.asarray(attr_all)
    a = np.copy(attr_all).reshape((-1, 3*9*3))
    b = np.unique(a, axis=0)
    ind = np.random.choice(np.arange(len(attr_all)), num_samples_keep,replace=False)
    # np.save(output_dir+'aR/'+str(i_a)+'_'+str(i_R), attr_all[ind])
    np.save(join(output_dir, 'aR', f"{i_a}_{i_R}"), attr_all[ind])
    
    print(str(i_R)+' -- '+str(dur)+' -- '+str(len(b) == len(a)))

0 -- 3.850550413131714 -- True
1 -- 3.8511228561401367 -- True
2 -- 3.871915340423584 -- True
3 -- 3.9785006046295166 -- True
4 -- 3.927070140838623 -- True
5 -- 3.885435104370117 -- True
6 -- 3.8904526233673096 -- True


In [ ]:
i_a = 2 
for i_R in range(7, 10):
    
    start_time = time.time()
    
    r_list = r_all_M10[i_R]
    ind_select = np.random.choice(len(r_list), num_samples_total, replace=False)

    attr_all = []

    for i_sample, i_r in tqdm(enumerate(ind_select)):

        # 1) x_size is determined by the rule 
        x_color = r_list[i_r] # e.g. [[1], [1], [1]]

        potential_list = []
        while len(potential_list) == 0: 
            # 2) sample x_size and x_color attribute values to be not any rule ~ 
            # to avoid shortcut by just looking at the first two panels, make first2 same as some rule 
            x_shape = sample_nan(r_all_M7, r_dict_M7, r_last_M7)
            x_size = sample_nan(r_all_M10, r_dict_M10, r_last_M10)

            # get the number of unique attribues per panel. This is the min for x_num
            num_attr_unique = [max([len(x_shape[i]), len(x_size[i]), len(x_color[i])]) for i in range(3)]

            # 3) sample number 
            # the way we sample it is to choose one 1)>x_min, 2) first 2 panel belong to some row 3) last panel not 
            potential_first2 = [k for k in r_dict_num_keylist if int(k[0])>=num_attr_unique[0] and int(k[1])>=num_attr_unique[1]]
            if len(potential_first2) > 0: 
                key_first2 = random.choice(potential_first2) # e.g., '77'
                d = r_dict_num[key_first2]
                potential_list = [i for i in range(num_attr_unique[2], 9) if i not in d]

        last = random.choice(potential_list) # could this give us no choice? 
        x_num = [[int(key_first2[0])], [int(key_first2[1])], [last]] # e.g., [7, 7, 4]
        x_pos = sample_nan_pos(x_num, r_dict_pos, r_dict_pos_withR) # 4) sample position 

        attr = np.asarray([fill_panel(x_pos[i], x_shape[i], x_size[i], x_color[i]) for i in range(3)])
        attr_all.append(attr)

        del x_shape, x_size, x_color, x_pos, x_num 

    dur = time.time() - start_time
    
    attr_all = np.asarray(attr_all)
    a = np.copy(attr_all).reshape((-1, 3*9*3))
    b = np.unique(a, axis=0)
    ind = np.random.choice(np.arange(len(attr_all)), num_samples_keep,replace=False)
    # np.save(output_dir+'aR/'+str(i_a)+'_'+str(i_R), attr_all[ind])
    np.save(join(output_dir, 'aR', f"{i_a}_{i_R}"), attr_all[ind])
    
    print(str(i_R)+' -- '+str(dur)+' -- '+str(len(b) == len(a)))

7 -- 4.129863262176514 -- True
8 -- 4.1532676219940186 -- True
9 -- 3.9504404067993164 -- True


### Attribute 3 Number

In [ ]:
i_a = 3
for i_R in range(7): 
    
    r_list = r_list_num[i_R]
    num_samples_per_r = int(np.ceil(num_samples_total/len(r_list)))

    attr_all = []

    for i_r in range(len(r_list)):

        start_time = time.time()

        for i_sample in trange(num_samples_per_r):

            x_num = r_list[i_r] # e.g., [[2], [2], [2]]

            x_shape = sample_x_with_num(x_num, r_all_M7, r_dict_M7, r_last_M7)
            x_size = sample_x_with_num(x_num, r_all_M10, r_dict_M10, r_last_M10)
            x_color = sample_x_with_num(x_num, r_all_M10, r_dict_M10, r_last_M10)

            x_pos = sample_nan_pos(x_num, r_dict_pos, r_dict_pos_withR) # 4) sample position

            attr = np.asarray([fill_panel(x_pos[i], x_shape[i], x_size[i], x_color[i]) for i in range(3)])
            attr_all.append(attr)

            del x_shape, x_size, x_color, x_pos, x_num 

    dur = time.time() - start_time
    
    attr_all = np.asarray(attr_all)
    a = np.copy(attr_all).reshape((-1, 3*9*3))
    b = np.unique(a, axis=0)
    ind = np.random.choice(np.arange(len(attr_all)), num_samples_keep,replace=False)
    # np.save(output_dir+'aR/'+str(i_a)+'_'+str(i_R), attr_all[ind])
    np.save(join(output_dir, 'aR', f"{i_a}_{i_R}"), attr_all[ind])
    
    print(str(i_R)+' -- '+str(dur)+' -- '+str(len(b) == len(a)))

0 -- 0.513812780380249 -- True
1 -- 0.8829183578491211 -- True
2 -- 0.591862678527832 -- True
3 -- 0.6046969890594482 -- True
4 -- 0.9286031723022461 -- True
5 -- 0.10526132583618164 -- True
6 -- 0.10176587104797363 -- True


### Attribute 4: number 

In [ ]:
r_all_num_all = []
for r_list in r_list_num: 
    for x in r_list: 
        r_all_num_all.append(x)

In [ ]:
i_a = 4
for i_R in range(7,10):
    
    start_time = time.time()
    
    r_list = r_list_pos[i_R-7]
    ind_select = np.random.choice(len(r_list), num_samples_total, replace=False)

    attr_all = []

    for i_sample, i_r in tqdm(enumerate(ind_select)):

        x_pos = r_list[i_r]
        x_num = [[len(x)] for x in x_pos]

        if x_num in r_all_num_all: 
            print('Error')

        x_shape = sample_x_with_num(x_num, r_all_M7, r_dict_M7, r_last_M7)
        x_size = sample_x_with_num(x_num, r_all_M10, r_dict_M10, r_last_M10)
        x_color = sample_x_with_num(x_num, r_all_M10, r_dict_M10, r_last_M10)
        attr = np.asarray([fill_panel(x_pos[i], x_shape[i], x_size[i], x_color[i]) for i in range(3)])

        attr_all.append(attr)
        del x_shape, x_size, x_color, x_pos, x_num 

    dur = time.time() - start_time
    
    attr_all = np.asarray(attr_all)
    a = np.copy(attr_all).reshape((-1, 3*9*3))
    b = np.unique(a, axis=0)
    ind = np.random.choice(np.arange(len(attr_all)), num_samples_keep,replace=False)
    # np.save(output_dir+'aR/'+str(i_a)+'_'+str(i_R), attr_all[ind])
    np.save(join(output_dir, 'aR', f"{i_a}_{i_R}"), attr_all[ind])
    
    print(str(i_R)+' -- '+str(dur)+' -- '+str(len(b) == len(a)))

7 -- 3.0972330570220947 -- True
8 -- 3.1581482887268066 -- True
9 -- 3.0713839530944824 -- True


In [1]:
datadir = r"/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Datasets/RPM_dataset/RPM1000k/aR"
import numpy as np 
import torch 
import os 
import pickle
import random 
import time 
from os.path import join
from tqdm import tqdm, trange
os.environ['CUDA_VISIBLE_DEVICES']='0'
device = torch.device('cpu')

In [4]:
aR_tuples = []
aR_tuples.extend([(a, R) for a in range(4) for R in range(10) ])
aR_tuples = aR_tuples[:37]
aR_tuples.extend([(4, 7), (4, 8), (4, 9)])

In [8]:
for a, R in aR_tuples: 
    print(f"Loading a={a}, R={R}")
    # make sure the data exist 
    assert os.path.exists(join(datadir, f"{a}_{R}.npy"))
    # data = np.load(join(datadir, f"{a}_{R}.npy"))
    # print(data.shape)
# aR_tuples

Loading a=0, R=0
Loading a=0, R=1
Loading a=0, R=2
Loading a=0, R=3
Loading a=0, R=4
Loading a=0, R=5
Loading a=0, R=6
Loading a=0, R=7
Loading a=0, R=8
Loading a=0, R=9
Loading a=1, R=0
Loading a=1, R=1
Loading a=1, R=2
Loading a=1, R=3
Loading a=1, R=4
Loading a=1, R=5
Loading a=1, R=6
Loading a=1, R=7
Loading a=1, R=8
Loading a=1, R=9
Loading a=2, R=0
Loading a=2, R=1
Loading a=2, R=2
Loading a=2, R=3
Loading a=2, R=4
Loading a=2, R=5
Loading a=2, R=6
Loading a=2, R=7
Loading a=2, R=8
Loading a=2, R=9
Loading a=3, R=0
Loading a=3, R=1
Loading a=3, R=2
Loading a=3, R=3
Loading a=3, R=4
Loading a=3, R=5
Loading a=3, R=6
Loading a=4, R=7
Loading a=4, R=8
Loading a=4, R=9


In [9]:
from os.path import join
from tqdm import tqdm

data_all = []
for a, R in tqdm(aR_tuples): 
    print(f"Loading a={a}, R={R}")
    # make sure the data exist 
    assert os.path.exists(join(datadir, f"{a}_{R}.npy"))
    data = np.load(join(datadir, f"{a}_{R}.npy"))
    data_all.append(data)
    # print(data.shape)
# aR_tuples

  0%|          | 0/40 [00:00<?, ?it/s]

Loading a=0, R=0


  2%|▎         | 1/40 [00:00<00:33,  1.16it/s]

Loading a=0, R=1


  5%|▌         | 2/40 [00:01<00:32,  1.16it/s]

...

 95%|█████████▌| 38/40 [00:33<00:01,  1.15it/s]

Loading a=4, R=8


 98%|█████████▊| 39/40 [00:34<00:00,  1.20it/s]

Loading a=4, R=9


100%|██████████| 40/40 [00:35<00:00,  1.14it/s]


In [10]:
attr_all = np.stack(data_all, axis=0)

In [11]:
np.save(join("/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Datasets/RPM_dataset/RPM1000k", 
             "attr_all_1000k.npy"), attr_all)